In [1]:
import pandas as pd

import techminer.data as data

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = data.load_scopus(df)

2020-06-05 17:26:59,029 - INFO - Renaming and selecting columns ...
2020-06-05 17:27:04,685 - INFO - Formatting author names ...
2020-06-05 17:27:04,690 - INFO - Disambiguating author names ...
2020-06-05 17:27:04,757 - INFO - Removing part of titles in foreing languages ...
2020-06-05 17:27:04,760 - INFO - Fusioning author and index keywords ...
2020-06-05 17:27:04,771 - INFO - NumExpr defaulting to 8 threads.
2020-06-05 17:27:04,789 - INFO - Extracting countries from affiliations ...
2020-06-05 17:27:10,855 - INFO - Extracting institutions from affiliations ...
2020-06-05 17:27:10,867 - INFO - Extracting country of 1st author ...
2020-06-05 17:27:10,870 - INFO - Extracting affiliation of 1st author ...
2020-06-05 17:27:10,874 - INFO - Counting number of authors ...


In [ ]:
data.coverage(df)

### Keywords

### FindString Dialog Box pag. 69

### Cleaning a list pag. 102

### Fuzzy match editor pag.126

### Stemming And / Stemming Or

### APPS

In [ ]:
import techminer.by_year as year
year.app(df)

In [ ]:
import techminer.by_term as term
term.app(df)

In [ ]:
import techminer.by_term_per_year as term_year
term_year.app(df)

In [ ]:
import techminer.co_occurrence as co_occurrence
co_occurrence.app(df)

In [2]:
import techminer.correlation as correlation
correlation.app(df)

AppLayout(children=(HTML(value="<h1>Correlation Analysis</h1><hr style='height:2px;border-width:0;color:gray;b…

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout

In [ ]:

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]

In [ ]:
FIGSIZE = (14, 10.0) ## Co-occurrente-analysis  <---- no tiene figura
FIGSIZE = (14, 9.0)

# NUEVA INTERFAZ

In [ ]:
from techminer.summary_by_term_per_year import documents_by_term_per_year, citations_by_term_per_year

In [ ]:
import techminer.plots as plt
from techminer.plots import COLORMAPS
from techminer.summary_by_term import citations_by_term, documents_by_term

WIDGET_WIDTH = "200px"
LEFT_PANEL_HEIGHT = "588px"
RIGHT_PANEL_WIDTH = "870px"
FIGSIZE = (14, 10.0)
PANE_HEIGHTS = ["80px", "650px", 0]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]



In [ ]:
#################
import ipywidgets as widgets
import numpy as np
from ipywidgets import AppLayout, Layout

from techminer.by_term import summary_by_term
from techminer.correlation import corr
from techminer.maps import Map
from techminer.plots import COLORMAPS

WIDGET_WIDTH = "200px"
LEFT_PANEL_HEIGHT = "650px"
RIGHT_PANEL_WIDTH = "870px"
FIGSIZE = (14, 10.0)
PANE_HEIGHTS = ["80px", "750px", 0]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]


#################


#
#
#  Correlation Analysis
#
#
def __body_0(x):
    #
    def server(**kwargs):
        #
        # Logic
        #
        column = kwargs["term"]
        by = kwargs["by"]
        method = kwargs["method"]
        min_link_value = float(kwargs["min_link_value"].split(" ")[0])
        cmap = kwargs["cmap"]
        filter_by = kwargs["filter_by"]
        filter_value = int(kwargs["filter_value"].split()[0])
        view = kwargs["view"]
        sort_by = kwargs["sort_by"]
        layout = kwargs["layout"]
        #
        #
        #
        s = summary_by_term(x, column)
        if filter_by == "Frequency":
            s = s[[column, "Num Documents"]]
            n_columns = len(s[s["Num Documents"] > filter_value])
            new_names = {
                a: "{} [{:d}]".format(a, b)
                for a, b in zip(s[column].tolist(), s["Num Documents"].tolist())
            }
        if filter_by == "Cited by":
            s = s[[column, "Cited by"]]
            n_columns = len(s[s["Cited by"] > filter_value])
            new_names = {
                a: "{} [{:d}]".format(a, b)
                for a, b in zip(s[column].tolist(), s["Cited by"].tolist())
            }
        #
        a = s[s.columns[1]].value_counts().sort_index(ascending=False)
        a = a.cumsum()
        a = a.sort_index(ascending=True)
        current_value = controls[4]["widget"].value
        controls[4]["widget"].options = [
            "{:d} [{:d}]".format(idx, w) for w, idx in zip(a, a.index)
        ]
        if current_value not in controls[4]["widget"].options:
            controls[4]["widget"].value = controls[4]["widget"].options[0]
        #
        #
        if view == "Matrix":
            controls[7]["widget"].disabled = False
            controls[8]["widget"].disabled = False
            controls[9]["widget"].disabled = True    
        if view == "Correlation map":
            controls[7]["widget"].disabled = False
            controls[8]["widget"].disabled = True
            controls[9]["widget"].disabled = False
        if view == "Chord diagram":
            controls[7]["widget"].disabled = False
            controls[8]["widget"].disabled = True
            controls[9]["widget"].disabled = True
        #
        #
        if n_columns > 50:
            controls[7]["widget"].disabled = True
            controls[8]["widget"].disabled = True
            output.clear_output()
            with output:
                display(widgets.HTML("<h3>Matrix exceeds the maximum shape</h3>"))
                return
        #
        #
        #
        matrix = corr(
            x,
            column=column,
            by=by,
            method=method,
            cmap=cmap,
            filter_by=filter_by,
            filter_value=filter_value,
            as_matrix=True,
            keywords=None,
        )
        #
        #
        #
        matrix = matrix.rename(columns=new_names, index=new_names)
        #
        output.clear_output()
        with output:
            if view == "Matrix":
                #
                # Sort order
                #
                if sort_by == "Frequency/Cited by asc.":
                    g = lambda m: int(m[m.find("[") + 1 : m.find("]")])
                    names = sorted(matrix.columns, key=g, reverse=False)
                    matrix = matrix.loc[names, names]
                if sort_by == "Frequency/Cited by desc.":
                    g = lambda m: int(m[m.find("[") + 1 : m.find("]")])
                    names = sorted(matrix.columns, key=g, reverse=True)
                    matrix = matrix.loc[names, names]
                if sort_by == "Alphabetic asc.":
                    matrix = matrix.sort_index(axis=0, ascending=True).sort_index(
                        axis=1, ascending=True
                    )
                if sort_by == "Alphabetic desc.":
                    matrix = matrix.sort_index(axis=0, ascending=False).sort_index(
                        axis=1, ascending=False
                    )
                #
                # View
                #
                display(
                    matrix.style.format(
                        lambda q: "{:+4.3f}".format(q) if q >= min_link_value else ""
                    ).background_gradient(cmap=cmap)
                )
                #
            if view == "Correlation map":
                #
                display(
                    correlation_map(
                        matrix=matrix,
                        layout=layout,
                        cmap=cmap,
                        figsize=(10, 10),
                        min_link_value=min_link_value,
                    )
                )
                #
            if view == "Chord diagram":
                #
                display(chord_diagram(matrix, cmap=cmap, minval=min_link_value))

    #
    # UI
    #
    controls = [
        # 0
        {
            "arg": "term",
            "desc": "Term to analyze:",
            "widget": widgets.Dropdown(
                options=[z for z in COLUMNS if z in x.columns],
                ensure_option=True,
                disabled=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        # 1
        {
            "arg": "by",
            "desc": "By Term:",
            "widget": widgets.Dropdown(
                options=[z for z in COLUMNS if z in x.columns],
                ensure_option=True,
                disabled=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        # 2
        {
            "arg": "method",
            "desc": "Method:",
            "widget": widgets.Dropdown(
                options=["pearson", "kendall", "spearman"],
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        # 3
        {
            "arg": "filter_by",
            "desc": "Filter by:",
            "widget": widgets.Dropdown(
                options=["Frequency", "Cited by"],
                disable=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        # 4
        {
            "arg": "filter_value",
            "desc": "Filter value:",
            "widget": widgets.Dropdown(
                options=[str(i) for i in range(10)],
                disable=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        # 5
        {
            "arg": "min_link_value",
            "desc": "Min link value:",
            "widget": widgets.Dropdown(
                options="-1.00 -0.25 0.00 0.125 0.250 0.375 0.500 0.625 0.750 0.875".split(
                    " "
                ),
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        # 6
        {
            "arg": "view",
            "desc": "View:",
            "widget": widgets.Dropdown(
                options=["Matrix", "Correlation map", "Chord diagram"],
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        # 7
        {
            "arg": "cmap",
            "desc": "Matrix colormap:",
            "widget": widgets.Dropdown(
                options=COLORMAPS,
                disable=False,
                layout=Layout(width=WIDGET_WIDTH),
                disabled=False,
            ),
        },
        # 8
        {
            "arg": "sort_by",
            "desc": "Sort order:",
            "widget": widgets.Dropdown(
                options=[
                    "Alphabetic asc.",
                    "Alphabetic desc.",
                    "Frequency/Cited by asc.",
                    "Frequency/Cited by desc.",
                ],
                disable=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        # 9
        {
            "arg": "layout",
            "desc": "Map layout:",
            "widget": widgets.Dropdown(
                options=[
                    "Circular",
                    "Kamada Kawai",
                    "Planar",
                    "Random",
                    "Spectral",
                    "Spring",
                    "Shell",
                ],
                disable=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
    ]
    #
    args = {control["arg"]: control["widget"] for control in controls}
    output = widgets.Output()
    with output:
        display(widgets.interactive_output(server, args,))
    return widgets.HBox(
        [
            widgets.VBox(
                [
                    widgets.VBox(
                        [widgets.Label(value=control["desc"]), control["widget"]]
                    )
                    for control in controls
                ],
                layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
            ),
            widgets.VBox(
                [output], layout=Layout(width=RIGHT_PANEL_WIDTH, align_items="baseline")
            ),
        ]
    )


#
#
# APP
#
#


def app(df):
    #
    body = widgets.Tab()
    body.children = [__body_0(df)]
    body.set_title(0, "Matrix")
    #
    return AppLayout(
        header=widgets.HTML(
            value="<h1>{}</h1><hr style='height:2px;border-width:0;color:gray;background-color:gray'>".format(
                "Correlation Analysis"
            )
        ),
        center=body,
        pane_heights=PANE_HEIGHTS,
    )


#####
app(df)

In [ ]:
import textwrap

import matplotlib.pyplot as pyplot
import numpy as np

from techminer.chord_diagram import ChordDiagram

TEXTLEN = 20

#############
def chord_diagram(
    x, figsize=(10, 10), cmap='Grays', alpha=1.0, minval=0.0, top_n_links=None, solid_lines=False
):
    """Creates a chord diagram from a correlation or an auto-correlation matrix.


    Examples
    ----------------------------------------------------------------------------------------------

    >>> df = pd.DataFrame(
    ...     {
    ...         'word 0': [1.00, 0.80, 0.70, 0.00,-0.30],
    ...         'word 1': [0.80, 1.00, 0.70, 0.50, 0.00],
    ...         'word 2': [0.70, 0.70, 1.00, 0.00, 0.00],
    ...         'word 3': [0.00, 0.50, 0.00, 1.00, 0.30],
    ...         'word 4': [-0.30, 0.00, 0.00, 0.30, 1.00],
    ...     },
    ...     index=['word {:d}'.format(i) for i in range(5)]
    ... )
    >>> df
            word 0  word 1  word 2  word 3  word 4
    word 0     1.0     0.8     0.7     0.0    -0.3
    word 1     0.8     1.0     0.7     0.5     0.0
    word 2     0.7     0.7     1.0     0.0     0.0
    word 3     0.0     0.5     0.0     1.0     0.3
    word 4    -0.3     0.0     0.0     0.3     1.0
    >>> fig = chord_diagram(df)
    >>> fig.savefig('sphinx/images/plotcd1.png')

    .. image:: images/plotcd1.png
        :width: 400px
        :align: center

    >>> fig = chord_diagram(df, top_n_links=5)
    >>> fig.savefig('sphinx/images/plotcd2.png')

    .. image:: images/plotcd2.png
        :width: 400px
        :align: center

    >>> fig = chord_diagram(df, solid_lines=True)
    >>> fig.savefig('sphinx/images/plotcd3.png')

    .. image:: images/plotcd3.png
        :width: 400px
        :align: center

    """

    x = x.copy()
    #
    
    # ---------------------------------------------------
    #
    # Node sizes
    #
    terms = x.columns
    node_sizes = [int(w[w.find("[") + 1 : w.find("]")]) for w in terms if "[" in w]
    if len(node_sizes) == 0:
        node_sizes = [10] * len(terms)
    else:
        max_size = max(node_sizes)
        min_size = min(node_sizes)
        if min_size == max_size:
            node_sizes = [300] * len(terms)
        else:
            node_sizes = [
                30 + int(50 * (w - min_size) / (max_size - min_size))
                for w in node_sizes
            ]

    #
    # Node colors
    #
    cmap = pyplot.cm.get_cmap(cmap)
    node_colors = [
        cmap(0.2 + 0.75 * node_sizes[i] / max(node_sizes))
        for i in range(len(node_sizes))
    ]
    #    
    # ---------------------------------------------------
    
    
    #
    cd = ChordDiagram()
    for idx, term in enumerate(x.columns):
        cd.add_node(term, color=node_colors[idx], s=node_sizes[idx])

    if top_n_links is not None and top_n_links <= len(x.columns):
        values = []
        for idx_col in range(len(x.columns) - 1):
            for idx_row in range(idx_col + 1, len(x.columns)):
                node_a = x.index[idx_row]
                node_b = x.columns[idx_col]
                value = x[node_b][node_a]
                values.append(value)
        values = sorted(values, reverse=True)
        minval = values[top_n_links - 1]

    style = ['-', '-', '--', ':']
    if solid_lines is True:
        style = list("----")

    width = [4, 2, 1, 1]
    if solid_lines is True:
        width = [4, 2, 1, 1]

    links = 0
    for idx_col in range(len(x.columns) - 1):
        for idx_row in range(idx_col + 1, len(x.columns)):

            node_a = x.index[idx_row]
            node_b = x.columns[idx_col]
            value = x[node_b][node_a]

            if value > 0.75 and value >= minval:
                cd.add_edge(
                    node_a,
                    node_b,
                    linestyle=style[0],
                    linewidth=width[0],
                    color="black",
                )
                links += 1
            elif value > 0.50 and value >= minval:
                cd.add_edge(
                    node_a,
                    node_b,
                    linestyle=style[1],
                    linewidth=width[1],
                    color="black",
                )
                links += 1
            elif value > 0.25 and value >= minval:
                cd.add_edge(
                    node_a,
                    node_b,
                    linestyle=style[2],
                    linewidth=width[2],
                    color="black",
                )
                links += 1
            elif value <= 0.25 and value >= minval and value > 0.0:
                cd.add_edge(
                    node_a, node_b, linestyle=style[3], linewidth=width[3], color="black",
                )
                links += 1

            if top_n_links is not None and links >= top_n_links:
                continue

    return cd.plot(figsize=figsize)



In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
interact(slow_function,i=widgets.FloatSlider(min=1e5, max=1e7, step=1e5, continuous_update=True));

In [ ]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return

In [ ]:
x = "hola"
x[:-1]

In [ ]:
import pandas as pd
x = pd.DataFrame({'a': [1, 2, 3, 4, 5], 'b': list('ABCDE')})

x.values.ravel().tolist()

In [ ]:
x

In [ ]:
x.values.shape

In [ ]:
print('hola')

In [ ]:
m = "abcdefg [1000]"
m[m.find('[')+1:m.find(']')]


In [ ]:
import networkx as nx
G = nx.petersen_graph()
nx.draw(G, with_labels=True, font_weight='bold')

In [ ]:
nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_circular(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_kamada_kawai(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_spectral(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_spring(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
[12]*5